https://dacon.io/competitions/official/235738/codeshare/2902?page=1&dtype=recent

In [9]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import datetime
import random
# import glob

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import scipy as sc
import scipy.cluster.hierarchy as shc
from scipy.stats import skew
from scipy.stats import spearmanr

from tqdm import tqdm
from glob import glob
from matplotlib import font_manager, rc, cm
plt.rcParams['font.family'] = 'Malgun Gothic'
%matplotlib inline

import sklearn
from sklearn.linear_model import *
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
# from sktime.forecasting.model_selection import temporal_train_test_split
# from sktime.utils.plotting import plot_series

import lightgbm as lgb
from lightgbm import LGBMRegressor

import catboost
from catboost import CatBoostRegressor
from catboost import Pool
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

SEED = 2
np.random.seed(SEED)

# 음성 데이터 분석용 라이브러리
import librosa
import librosa.display

In [56]:
data_path = r'D:\Study\Competition_English_Sound_Classification\Data'

In [60]:
africa_train_paths = glob(data_path + r'\train\africa\*.wav')
australia_train_paths = glob(data_path + r'\train\australia\*.wav')
canada_train_paths = glob(data_path + r'\train\canada\*.wav')
england_train_paths = glob(data_path + r'\train\england\*.wav')
hongkong_train_paths = glob(data_path + r'\train\hongkong\*.wav')
us_train_paths = glob(data_path + r'\train\us\*.wav')

path_list = [africa_train_paths,
             australia_train_paths,
             canada_train_paths,
             england_train_paths,
             hongkong_train_paths,
             us_train_paths]

In [65]:
test_paths = glob(data_path + r'\test\*.wav')

In [68]:
def load_data(paths):
    result = []
    
    for path in tqdm(paths):
        # sr(sampling rate) - 1초당 16000개의 데이터를 샘플링 한다는 의미
        data, sr = librosa.load(path, sr = 16000)
        result.append(data)
        
    result = np.array(result)
    
    return result

def get_features(data, sr = 16000, n_fft = 2048, win_length = 200, hop_length = 160, n_mels = 64):
    mel = []
    for i in tqdm(data):
        mel_ = librosa.feature.melspectrogram(i,
                                              sr = sr,
                                              n_fft = n_fft,
                                              win_length = win_length,
                                              hop_length = hop_length,
                                              n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)
    
    return mel

def set_length(data, d_mini):
    result = []
    for value in tqdm(data):
        value = value[:d_mini]
        if len(value) < d_mini:
            value = np.append(value, [0] * (d_mini - len(value)))
            
        result.append(value)
    
    result = np.array(resutl)
    
    return result

### 작업의 편의성을 위해 데이터를 numpy 파일로 변환하여 저장

In [72]:
if os.path.isdir(os.path.join(data_path, 'npy_data')):
    pass
else:
    os.mkdir(os.path.join(data_path, 'npy_data'))

In [73]:
africa_train_data = load_data(africa_train_paths)
# np.save(os.path.join(data_path, 'africa.npy', africa_train_data)

australia_train_data = load_data(australia_train_paths)
# np.save("./dataset/australia-sorted.npy", australia_train_data)

canada_train_data = load_data(canada_train_paths)
# np.save("./dataset/canada-sorted.npy", canada_train_data)

england_train_data = load_data(england_train_paths)
# np.save("./dataset/england-sorted.npy", england_train_data)

hongkong_train_data = load_data(hongkong_train_paths)
# np.save("./dataset/hongkong-sorted.npy", hongkong_train_data)

us_train_data = load_data(us_train_paths)
# np.save("./dataset/us-sorted.npy", us_train_data)
# train_data_list = [africa_train_data, australia_train_data, canada_train_data, england_train_data, hongkong_train_data, us_train_data]

100%|██████████| 10000/10000 [34:17<00:00,  4.86it/s] 


In [76]:
test_data = load_data(test_paths)

100%|██████████| 6100/6100 [18:55<00:00,  5.37it/s] 


In [77]:
# np.save(os.path.join(data_path, r'npy_data/africa.npy'), africa_train_data)
# np.save(os.path.join(data_path, r'npy_data/australia.npy'), australia_train_data)
# np.save(os.path.join(data_path, r'npy_data/canada.npy'), canada_train_data)
# np.save(os.path.join(data_path, r'npy_data/england.npy'), england_train_data)
# np.save(os.path.join(data_path, r'npy_data/hongkong.npy'), hongkong_train_data)
# np.save(os.path.join(data_path, r'npy_data/us.npy'), us_train_data)

np.save(os.path.join(data_path, r'npy_data/test.npy'), test_data)

In [ ]:
np.load(os.path.join(data_path, '/npy_data/africa.npy'), allow_pickle = True)

### Data Reshape

In [ ]:
X_train = np.concatenate(train_data_list, axis = 0)
X_train = set_length(X_train, 100000)

X_train_200 = get_feature(data = X_train, win_length = 200)
X_train_400 = get_feature(data = X_train, win_length = 400)
X_train_800 = get_feature(data = X_train, win_length = 800)
X_train_1000 = get_feature(data = X_train, win_length = 1000)

# 
X_train_200 = X_train_200.reshape(X_train_200.shape[0], X_train_200.shape[1], X_train_200.shape[2], 1)
X_train_400 = X_train_400.reshape(X_train_400.shape[0], X_train_400.shape[1], X_train_400.shape[2], 1)
X_train_800 = X_train_800.reshape(X_train_800.shape[0], X_train_800.shape[1], X_train_800.shape[2], 1)
X_train_1000 = X_train_1000.reshape(X_train_1000.shape[0], X_train_1000.shape[1], X_train_1000.shape[2], 1)

X_train_multi = np.concatenate([X_train_200,
                                X_train_400,
                                X_train_800,
                                X_train_1000], -1)
np.save(os.path.join(data_path, r'npy_data/X_train_multi.npy'), X_train_multi)

In [ ]:
X_test = np.load(os.path.join(data_path, r'npy_data/test.npy'), allow_pickle = True)
X_test = set_length(X_test, 100000)

X_test_200 = get_feature(data = X_test, win_length = 200)
X_test_400 = get_feature(data = X_test, win_length = 400)
X_test_800 = get_feature(data = X_test, win_length = 800)
X_test_1000 = get_feature(data = X_test, win_length = 1000)

X_test_200 = X_test_200.reshape(X_test_200.shape[0], X_test_200.shape[1], X_test_200.shape[2], 1)
X_test_400 = X_test_400.reshape(X_test_400.shape[0], X_test_400.shape[1], X_test_400.shape[2], 1)
X_test_800 = X_test_800.reshape(X_test_800.shape[0], X_test_800.shape[1], X_test_800.shape[2], 1)
X_test_1000 = X_test_1000.reshape(X_test_1000.shape[0], X_test_1000.shape[1], X_test_1000.shape[2], 1)
X_test_multi = np.concatenate([X_test_200,
                               X_test_400,
                               X_test_800,
                               X_test_1000], axis = 1)
np.save(os.path.join(data_path, r'npy_data/X_test_multi.npy'), X_test_multi)

# Network & DataLoader

- 네트워크는 Conv2d, BatchNorm2d, ReLU 를 합친 block으로 주로 구성되어있고, output 전 dropout, AvgPool, AdaptiveAvgPool을 적용했습니다.
- 데이터를 배치단위로 불러올 때 train 데이터의 분포로 min-max scaling 해주었습니다.
- 1/2 확률로 데이터를 rolling 하여 augmentation 효과를 노려봤지만 큰 도움이 되지는 않았던거 같아서 이것은 적용하지 않았습니다.